In [ ]:
# import files and libraries

import generative_breastCancer_model as gm
import numpy as np
import pandas as pd

ct_interactions = np.genfromtxt("/Users/bp/data/cellular_automata_bio394/ct_interactions_knn8.csv",delimiter=",",skip_header=True)
total_freq = np.genfromtxt("/Users/bp/data/cellular_automata_bio394/total_freq.csv",delimiter=",")

In [ ]:
# set dimension

dimx, dimy = 55,55
cellsize=1000/dimx

In [ ]:
e = gm.Board(dimx,dimy,transition_matrix=ct_interactions,total_freq = total_freq,cellsize=cellsize)

In [ ]:
gm.generate(e,plot=True,generate_func=gm.generate_conv)

In [ ]:
def open_neighbors(cells,x,y):    
    #compute possible moves at position x,y in cells.state    
    moves = np.array([[[-1,-1],[-1,0],[-1,1]],
                 [[0,-1],[0,0],[0,1]],
                 [[1,-1],[1,0],[1,1]]])

    left_bound = x-1
    left = 0
    right_bound = x+2
    right = 3
    upper_bound = y-1
    upper = 0
    lower_bound = y+2
    lower = 3
    #check if at right or left boundary
    if x == cells.dimx-1:
        right_bound = x+1
        right -=1
    elif x == 0:
        left_bound = 0
        left +=1
    #check if at upper or lower boundary
    if y == cells.dimy-1:
        lower_bound = y+1
        lower -=1
    elif y == 0:
        upper_bound = 0
        upper +=1
    
    neig = cells.processed[upper_bound:lower_bound,left_bound:right_bound]
    moves = moves[upper:lower,left:right]
    open_pos = neig == 0
    possible_moves = moves[open_pos]
    
    #no possible moves
    if not possible_moves.any():
        return []
    else:
        return possible_moves

In [ ]:
def best_start(cells):

    kernel = [[1,1,1],
              [1,0,1],
              [1,1,1]]
    defined_positions = np.array(cells.processed!=0,dtype=int)
    neig_count = conv(defined_positions,kernel,mode="constant",cval=0)
    #only consider unused spaces
    neig_count[cells.processed!=0] = -1
    #max number of neigbors
    maxi = np.max(neig_count)
    #if -1 then no open positions left
    if maxi == -1:
        return -1,-1
    #consider only positions with the most amount of information given
    open_pos = np.argwhere(neig_count==maxi)
    #choose one of the neighbors randomly and continue walk
    new_y,new_x = open_pos[np.random.randint(0,len(open_pos))]
   
    return new_y,new_x

In [ ]:
def get_pde():

In [ ]:
def sample():

In [ ]:
# todo: add estimation of mu and sigma for distance matrix
# todo: implement networkX

def spread_spatial(cells):
    # choose random start
    y_start,x_start = np.random.choice(cells.states.shape[0]),np.random.choice(cells.states.shape[1])
    metacluster = cells.states[y_start,x_start]
    
    # create array to keep track which cells where already processed (=1)
    cells.processed = np.zeros(cells.states.shape)
    
    # initiate pandas dataframe
    dic = {"x": [x_start], "y":[y_start], "metacluster":[metacluster]}
    graph_df = pd.DataFrame(dic)
    cells.processed[y_start,x_start] = 1
    
    x,y = x_start,y_start
    while len(np.argwhere(cells.processed == 0)) > 0:
        possible_moves = open_neighbors(cells,x,y)
        #if no moves possible from position x,y then search for new position
        if len(possible_moves) == 0:
            new_y,new_x = best_start(cells)
    
        #if no moves possible with best_start we are finished
            if new_y == -1:
                break
        else:    
            #choose one of the possible moves
            delta_y, delta_x = possible_moves[np.random.randint(0,len(possible_moves))]
            
            #compute new position
            new_x, new_y = x+delta_x, y+delta_y
            
        #assign new position value according to transition matrix
        #take all spatial neighbors into account
        neig = neighbors(cells, new_x, new_y)

        # todo: create distribution and sample from it
        pdf = np.zeros()
        for s in neig:
            if s == 0:
                continue
            else:
                pdf += get_pde()
                count_neig += 1
                
        # sampling of new_x and new_y
        new_x,new_y = sample(pdf)
        
        #update current state and value
        x,y = new_x,new_y
        state = cells.states[y,x]
        
        # insert processed cell
        graph_df.loc[len(df.index)] = [x,y,state]
    